## Robot@Home 2 - Scenes


`R@H2 notebook series. Version 1.1`



### Scenes



Scenes contain the 3D coloured point cloud reconstruction for each room. Such
reconstruction was built employing a mapping application which registers the
RGB-D observations employing different ICP-based techniques. The 3D point clouds
are stored as plain text files containing the coordinates and colors of the
points that make up the 3D cloud. These files can be easily visualized with the
built-in function or a software like [MeshLab](https://www.meshlab.net/).

![img](./70-scenes-meshlab.png "Anto's livingroom (12-scene.txt) displayed thorugh MeshLab")



In [1]:
import os
import pandas as pd
from robotathome import RobotAtHome
from robotathome import logger, log, set_log_level
from robotathome import time_win2unixepoch, time_unixepoch2win
from robotathome import plot_scene

In [2]:
log.set_log_level('INFO')  # SUCCESS is the default

my_rh_path = '~/WORKSPACE/R@H2-2.0.1'
my_rgbd_path = os.path.join(my_rh_path, 'files/rgbd')
my_scene_path = os.path.join(my_rh_path, 'files/scene')
my_wspc_path = '~/WORKSPACE'

try:
    rh = RobotAtHome(my_rh_path, my_rgbd_path, my_scene_path, my_wspc_path)
except:
    logger.error("Something was wrong")

2022-01-23 21:48:38.309 | SUCCESS  | robotathome.core.reader:__open_dataset:85 - Connection is established: rh.db


As usual, this toolbox provides a function to get scenes



In [3]:
scenes = rh.get_scenes()
logger.info("\nScenes: \n{}", scenes)

2022-01-23 21:48:38.325 | INFO     | __main__:<module>:2 - 
Scenes: 
    id  home_session_id  home_subsession_id  home_id  room_id  \
0    0                0                   0        0        0   
1    1                0                   0        0        1   
2    2                0                   0        0        3   
3    3                0                   0        0        4   
4    4                0                   0        0        5   
..  ..              ...                 ...      ...      ...   
67  67                6                   1        4       36   
68  68                6                   0        4       39   
69  69                6                   1        4       39   
70  70                6                   0        4       40   
71  71                6                   1        4       40   

                                           scene_file  
0   /home/goyo/WORKSPACE/R@H2-2.0.1/files/scene/se...  
1   /home/goyo/WORKSPACE/R@H2-2.0.1/fi

After some notebooks we have enough skills to advance with an example of multi
column filtering



In [4]:
# Let's suppose we want to get the scene file of the kitchen of the home
# session alma-s1

# Get the ids from their names
hs_id = rh.name2id('anto-s1','hs')
r_id = rh.name2id('anto_livingroom1','r')

# Let's query the scene dataframe with the ids
scene =  scenes.query(f'home_session_id=={hs_id} & room_id=={r_id}')
# Extract the value of the scene_file field
scene_file = scene.scene_file.values[0]
logger.info(f'\nScene file: \n{scene_file}')

2022-01-23 21:48:38.346 | INFO     | __main__:<module>:12 - 
Scene file: 
/home/goyo/WORKSPACE/R@H2-2.0.1/files/scene/session_2/anto/livingroom1/subsession_1/12_scene.txt


R@H2 also provides a function to interactively plot the scene. You will need to
install Open3D to use it.

    pip install open3d

Once installed run the following function



In [5]:
plot_scene(scene_file)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


![img](./70-scenes-plot-scene.png "Screenshot of the plot<sub>scene</sub> output")



### Labeled 3D point maps



Each reconstructed room, i.e. scene has been labeled setting bounding boxes to
the objects appearing in the point cloud reconstruction and including annotations
with the ground truth information about their category, e.g. counter, book,
couch, shelf, as well as an object id to identify the particular instance, i.e.
counter-1, book-3, etc.

To get the labels for a scene use the following function



In [6]:
# let's extract the labels for any scene, for example the scene with id = 0
id = scene['id'].values[0]
scene_labels = rh.get_scene_labels(id)
logger.info(f'\nScene labels: {scene_labels.info()}')

2022-01-23 21:49:01.941 | INFO     | __main__:<module>:4 - 
Scene labels: None


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85 entries, 0 to 84
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             85 non-null     int64  
 1   local_id       85 non-null     int64  
 2   scene_id       85 non-null     int64  
 3   object_id      85 non-null     int64  
 4   object_name    85 non-null     object 
 5   bb_pose_x      85 non-null     float64
 6   bb_pose_y      85 non-null     float64
 7   bb_pose_z      85 non-null     float64
 8   bb_pose_yaw    85 non-null     float64
 9   bb_pose_pitch  85 non-null     float64
 10  bb_pose_roll   85 non-null     float64
 11  bb_corner1_x   85 non-null     float64
 12  bb_corner1_y   85 non-null     float64
 13  bb_corner1_z   85 non-null     float64
 14  bb_corner2_x   85 non-null     float64
 15  bb_corner2_y   85 non-null     float64
 16  bb_corner2_z   85 non-null     float64
dtypes: float64(12), int64(4), object(1)
memory usage: 11.4+ 

As you can see, for each label there is data about its pose and 3D coordinates.



In [7]:
logger.info(f'\nThere are {len(scene_labels)} object labels in the scene {id}\n')
# Let's examine the first bounding box
local_id = 0
logger.info('\nData about the label {}', scene_labels.query('local_id==0').loc[0,:])

2022-01-23 21:49:01.948 | INFO     | __main__:<module>:1 - 
There are 85 object labels in the scene 12

2022-01-23 21:49:01.953 | INFO     | __main__:<module>:4 - 
Data about the label id                     386
local_id                 0
scene_id                12
object_id              369
object_name      cushion_0
bb_pose_x          1.45029
bb_pose_y        -0.618376
bb_pose_z         0.537995
bb_pose_yaw        0.21677
bb_pose_pitch    -0.187421
bb_pose_roll     -0.407209
bb_corner1_x       1.27315
bb_corner1_y      -0.92887
bb_corner1_z      0.444868
bb_corner2_x       1.65392
bb_corner2_y      -0.34463
bb_corner2_z      0.536396
Name: 0, dtype: object


Each box delimits an object for which the dataset has additional information
about its features. If you consider this information of interest you can obtain
it by accessing the extended set of labels. Please, note that this extended set
is experimental: object info has been obtained from 2D observations and the
bounding box set has slightly more records (#2446) than the objects one (#2125).



In [8]:
# let's extract the labels for any scene, for example the scene with id = 0
id = scene['id'].values[0]
scene_labels = rh.get_scene_labels(id, True)
logger.info(f'\nScene labels: {scene_labels.info()}')

2022-01-23 21:49:01.980 | INFO     | __main__:<module>:4 - 
Scene labels: None


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85 entries, 0 to 84
Data columns (total 50 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      85 non-null     int64  
 1   local_id                85 non-null     int64  
 2   scene_id                85 non-null     int64  
 3   object_id               85 non-null     int64  
 4   object_name             85 non-null     object 
 5   object_type_id          85 non-null     int64  
 6   bb_pose_x               85 non-null     float64
 7   bb_pose_y               85 non-null     float64
 8   bb_pose_z               85 non-null     float64
 9   bb_pose_yaw             85 non-null     float64
 10  bb_pose_pitch           85 non-null     float64
 11  bb_pose_roll            85 non-null     float64
 12  bb_corner1_x            85 non-null     float64
 13  bb_corner1_y            85 non-null     float64
 14  bb_corner1_z            85 non-null     floa

Continuing the example



In [9]:
logger.info(f'\nThere are {len(scene_labels)} object labels in the scene {id}\n')
# Let's examine again the first bounding box
logger.info('\nData about the label \n{}', scene_labels.query('local_id==0').loc[0,:])

2022-01-23 21:49:01.986 | INFO     | __main__:<module>:1 - 
There are 85 object labels in the scene 12

2022-01-23 21:49:01.994 | INFO     | __main__:<module>:3 - 
Data about the label 
id                              386
local_id                          0
scene_id                         12
object_id                       369
object_name               cushion_0
object_type_id                   19
bb_pose_x                   1.45029
bb_pose_y                 -0.618376
bb_pose_z                  0.537995
bb_pose_yaw                 0.21677
bb_pose_pitch             -0.187421
bb_pose_roll              -0.407209
bb_corner1_x                1.27315
bb_corner1_y               -0.92887
bb_corner1_z               0.444868
bb_corner2_x                1.65392
bb_corner2_y               -0.34463
bb_corner2_z               0.536396
planarity                  0.371469
scatter                    0.579293
linearity                  0.183205
min_height                  0.33062
max_height            